Imports and chdir for non-ARM processors

In [ ]:
import os
os.chdir('/Users/benjaminshih/opt/miniconda3/envs/filer/lib/python3.8/site-packages')
from jupyter_dash import JupyterDash

In [ ]:
import requests
import matplotlib as mpl
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import plotly.express as px
import dash
import numpy as np
from dash import dcc
from dash import  html
import plotly.graph_objects as go

%matplotlib inline
%config InlineBackend.figure_format='retina'

#aesthetic params
nord = ['8fbcbb', '81a1c1', 'bf616a', 'd08770', 'ebcb8b', 'a3be8c', 'b48ead', 'd8dee9']
nord_hex = ["#"+i for i in nord]
custom_colors = mpl.cycler('color', nord)
mpl.rcParams['axes.prop_cycle'] = custom_colors
mpl.rcParams['figure.dpi'] = 300

rc('font',**{'family':'serif','serif':['Times']})

In [ ]:
#global vars
FILER_URL= 'https://tf.lisanwanglab.org/FILER'

data_region = 'get_data_region.php' 
metadata = 'get_metadata.php'
overlap_track = 'get_overlapping_tracks_by_coord.php'

In [ ]:
def request_data(base_url: str, endpoint: str, params: dict) -> "JSON":
    """
    Returns the JSON data for a particular endpoint (one of track data, metadata or overlap data) and 
    particular parameters

    Parameters:
        base_url (str): FILER or GADB base URL to fetch from
        endpoint (str): endpoint to fetch from, either data region, metadata, or overlapping tracks
        params (dict): request-specific parameters
        
    Returns:
        JSON dictionary containing API call data
    """
    r = requests.get(base_url + "/" + endpoint, params=params)
    data = r.json()
    
    if r.status_code != requests.codes.ok:
        print("Request failed")
        return
    
    return data

### Metadata Example

Generate HTML Heatmap with hover information from FILER API dataframe:
- Demonstrates ability to quickly query and return visualizations of data within FILER database

In [ ]:
data = request_data(FILER_URL, metadata, 
                    {'genomeBuild':'hg19', 'dataSource':'ROADMAP', 'assayType':'ChIP-seq'})
df = pd.DataFrame(data)
df_metadata = df
df = df[["cell type", "system category", "Number of intervals"]]

#fig = px.density_heatmap(df, x="system category", y="cell type", marginal_x="histogram", marginal_y="histogram", color_continuous_scale=nord_hex)
#fig.write_html("test.html", auto_open=True)

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)

In [ ]:
data = request_data(FILER_URL, overlap_track, {'genomeBuild':'hg19', 'region':'chr1:1103243-1103243', 
                                               'filterString':'.', 'outputFormat':'json'})
df = pd.DataFrame(data)
df = df[df.columns.difference(["wget command", "tabix_index Download", "Link out URL", "File name", "Raw File URL", "Raw file download", "Raw file md5", "Processed file md5", \
                               "Processed File Download URL", "filepath", "File format", "File size", "Downloaded date", "Date added to GADB", "Release date"])]
df = df[["Data Category", "system category", "bp covered"]]
fig = px.density_heatmap(df, x="system category", y="Data Category", color_continuous_scale=nord_hex)
fig.write_html("test2.html", auto_open=True)

In [ ]:
data = request_data(FILER_URL, overlap_track, {'genomeBuild':'hg19', 'region':'chr1:1103243-1103243', 
                                               'filterString':'.', 'outputFormat':'json'})
df = pd.DataFrame(data)
df = df[df.columns.difference(["wget command", "tabix_index Download", "Link out URL", "File name", "Raw File URL", "Raw file download", "Raw file md5", "Processed file md5", \
                               "Processed File Download URL", "filepath", "File format", "File size", "Downloaded date", "Date added to GADB", "Release date"])]
df = df[["Data Category", "system category", "bp covered"]]
fig = px.histogram(df, x="Data Category", y="bp covered", 
                   marginal="box",
                   hover_data=['system category'])
fig.write_html("test3.html", auto_open=True)

#Error in ```dash``` module for interactivity (dropdowns, etc.), will debug env

In [ ]:
fig = go.Figure(
    data=[go.Bar(y=[2, 1, 3])],
    layout_title_text="Native Plotly rendering in Dash"
)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig),
])

app.run_server()

In [ ]:
app = dash.Dash(__name__)

data = df_metadata
x_default = data['system category']
y_default = data['cell type']


app.layout = html.Div([
    html.H1('Metadata Analytics'),
    html.Div([
        html.Div([
            html.H4('Select X'),
            dcc.Dropdown(
                id='x_dropdown',
                options=[{'label': i, 'value': i} for i in list(data.columns)],
                value = x_default
            ),
          
        ],
        style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            html.H4('Select Y'),
            dcc.Dropdown(
                id='y_dropdown',
                options=[{'label': i, 'value': i} for i in list(data.columns)],
                value = y_default
            ),
        
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
        dcc.Graph(id='heatmap',  
        figure = {
            'data': [go.Heatmap(
                x=x_default,
                y=y_default,
                name = 'first legend group',
                colorscale='Viridis')],
            'layout': go.Layout(
                xaxis = dict(title = "X"),
                yaxis = dict(title = "Y"),
              )

        })
    ]),

  
])
@app.callback(
    dash.dependencies.Output(component_id='y_dropdown',component_property='options'),
    [dash.dependencies.Input(component_id='x_dropdown',component_property='value')]
)


def update_Menu_dropdown(selected_POS):
    return [{'label': i, 'value': i} for i in list(data.columns)]

@app.callback(
    dash.dependencies.Output(component_id='heatmap',component_property='figure'),
    [dash.dependencies.Input(component_id='x_dropdown',component_property='value'),
    dash.dependencies.Input(component_id='y_dropdown',component_property='value')]
)

def update_graph(x_dropdown, y_dropdown):
    return {
        'data': [go.Heatmap(
                x=data[x_dropdown],
                y=data[y_dropdown],
                z=data["Number of intervals"],
                xgap = 2,
                ygap = 2,
                colorscale='Viridis')],
        'layout': go.Layout()
    }

app.run_server(debug=True, use_reloader=False)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

import pandas as pd

app = dash.Dash(__name__)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)